# Final ETL Load
In this file we will load the resulting csv file to a MySQL database

In [1]:
#We will start by importing all the dependencies

import pandas as pd

from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base

from config import dbuser, dbpasswd, dburi, dbport, dbname

Base = declarative_base()

In [2]:
#We load the file
file =  pd.read_csv("../Clean/final_version/final_database.csv", delimiter=";")

In [3]:
# And transform the file to a pandas dataframe
db = pd.DataFrame(file)
db.head()

,Año,Mes,Alcaldía,Categoría de delito,Delito,Unidad de investigación,Fecha inicio,Colonia,Geopoint
0,2018,Octubre,ALVARO OBREGON,DELITO DE BAJO IMPACTO,ABUSO DE CONFIANZA,UI-3CD,2018-10-17 13:00:24,LAS AGUILAS 3ER PARQUE,"19.35177196, -99.220583382"
1,2018,Octubre,AZCAPOTZALCO,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,ROBO DE VEHICULO DE SERVICIO PÚBLICO CON VIOLE...,UI-3SD,2018-10-17 13:03:39,EL ROSARIO,"19.5085668561, -99.2083000919"
2,2018,Octubre,GUSTAVO A MADERO,DELITO DE BAJO IMPACTO,ROBO DE OBJETOS,UI-3CD,2018-10-17 13:05:09,GUADALUPE TEPEYAC,"19.4663394581, -99.1243321831"
3,2018,Octubre,GUSTAVO A MADERO,DELITO DE BAJO IMPACTO,ABUSO DE CONFIANZA,UI-3SD,2018-10-17 13:09:04,EMILIANO ZAPATA,"19.4628200485, -99.1163214933"
4,2018,Octubre,MIGUEL HIDALGO,ROBO A REPARTIDOR CON Y SIN VIOLENCIA,ROBO A REPARTIDOR SIN VIOLENCIA,UI-1SD,2018-10-17 13:09:22,PERIODISTA,"19.4503050199, -99.2199589505"


In [4]:
#We create a class that will represent the table from the MySQL table
class delitos(Base):
    __tablename__="delitos"
    id = Column(Integer, primary_key=True)
    Año = Column(Integer)
    Mes = Column(String)
    Alcaldia = Column(String)
    Categoria_del_delito = Column(String)
    Delito = Column(String)
    Unidad_de_inestigacion = Column(String)
    Fecha_inicio = Column(String)
    Colonia = Column(String)
    Geopoint = Column(String)
    Zona_patrullaje = Column(String)

In [5]:
#Inspect the table we created
Base.metadata.tables

immutabledict({'delitos': Table('delitos', MetaData(bind=None), Column('id', Integer(), table=<delitos>, primary_key=True, nullable=False), Column('Año', Integer(), table=<delitos>), Column('Mes', String(), table=<delitos>), Column('Alcaldia', String(), table=<delitos>), Column('Categoria_del_delito', String(), table=<delitos>), Column('Delito', String(), table=<delitos>), Column('Unidad_de_inestigacion', String(), table=<delitos>), Column('Fecha_inicio', String(), table=<delitos>), Column('Colonia', String(), table=<delitos>), Column('Geopoint', String(), table=<delitos>), schema=None)})

In [6]:
#Start engine to our MySQL database
engine = create_engine(f"mysql://{dbuser}:{dbpasswd}@{dburi}:{dbport}/{dbname}")

In [7]:
#Create but first checks if it allready exists
Base.metadata.create_all(engine, checkfirst=True)

In [8]:
#We start the session
from sqlalchemy.orm import Session
session = Session(engine)

In [9]:
#We iterate across the local dataframe, for each row we add it to the session
for row in db.iterrows() :
    session.add(delitos(Año=row[1][0],Mes=row[1][1],Alcaldia=row[1][2],Categoria_del_delito=row[1][3],
                        Delito=row[1][4],Unidad_de_inestigacion=row[1][5],Fecha_inicio=row[1][6],
                       Colonia=row[1][7],Geopoint=row[1][8],Zona_patrullaje=row[1][9]))
   #print(row[1][0])

In [10]:
#We commit the information to the database
session.commit()